In [1]:
# installations des bibliothèques
!pip install geemap --upgrade
!pip install earthengine-api --upgrade

In [2]:
# import des packages
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# VARIABLES DE CONFIGURATION
NOM_DU_PROJET = 'initialisation-478314'
ASSET_PREFIX  = f'projects/{NOM_DU_PROJET}/assets/'

# --- Authentification & Initialisation
ee.Authenticate()                                # une seule fois par session
ee.Initialize(project=NOM_DU_PROJET)

print("GEE initialisé avec succès.")

Enter verification code:  4/1ASc3gC2ed5g_lVoyMk-gGs1OZvN2pCSbn3vEXZ__ZV7eUoKEtiIpjUmJGp4



Successfully saved authorization token.
GEE initialisé avec succès.


In [8]:
# Charger la collection ERA5 DAILY
era5 = ee.ImageCollection("ECMWF/ERA5/DAILY")

# Sélectionner la bande total_precipitation
precipitation = era5.select('total_precipitation')

# Définir la géométrie du Ghana
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
ghana = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))
geometry = ghana.geometry()

# Restreindre à la période 2020-2025
precipitation_filtered = precipitation.filterDate('2020-01-01', '2025-12-31') \
                                     .filterBounds(geometry)

# Convertir en précipitations annuelles moyennes
# Grouper par année et calculer la somme annuelle
years = ee.List.sequence(2020, 2025)

def annual_precipitation(year):
    year = ee.Number(year)
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)
    
    # Somme des précipitations journalières pour l'année
    annual = precipitation_filtered.filterDate(start, end) \
                                   .sum() \
                                   .clip(geometry)
    
    return annual.set('year', year)

# Créer collection de précipitations annuelles
annual_precip = ee.ImageCollection.fromImages(
    years.map(annual_precipitation)
)

# Calculer la moyenne inter-annuelle (2020-2025)
mean_annual_precip = annual_precip.mean().clip(geometry)